In [1]:
import sys 
sys.path.append('../../')
from src.import_packages import *

from bs4 import BeautifulSoup
import requests

Importing the following packages: pandas, numpy, matplotlib, seaborn, plotly, warnings


### Seasons

In [2]:
url = 'https://southpark.fandom.com/wiki/Portal:Scripts'
page = requests.get(url)
soup = BeautifulSoup(page.text)

In [19]:
seasons = {}

for enum, i in enumerate(soup.find_all('div', class_ = 'wikia-gallery-item')):
    seasons[enum + 1] = 'https://southpark.fandom.com' + i.find('a', href = True)['href']

### Episodes within seasons

In [41]:
seasons_episodes = {}

for i in seasons:
    
    page = requests.get(seasons[i])
    soup = BeautifulSoup(page.text)

    episodes = {}

    for enum, j in enumerate(soup.find_all('div', class_ = 'wikia-gallery-item')):
        episodes[enum + 1] = 'https://southpark.fandom.com' + j.find('a', href = True)['href']

    seasons_episodes[i] = episodes

In [42]:
len(seasons_episodes)

24

### Transcripts

In [101]:
collector = []

for i in seasons_episodes:

    for j in seasons_episodes[i]:

        _link = seasons_episodes[i][j]

        page = requests.get(_link)
        soup = BeautifulSoup(page.text)

        _title = soup.find('h1', id = 'firstHeading').text.strip().split('/')[0]
        _table = pd.read_html(str(soup.find_all('table', class_ = 'headerscontent')[1]))[0]

        if _table.shape[1] == 2:
            _table.columns = ['speaker', 'line']

        if _table.shape[1] == 3:
            _table.columns = ['speaker', 'line', 'extra']

        _table['season'] = i
        _table['episode_num'] = j
        _table['episode_title'] = _title

        collector.append(_table)

### Concatenate transcripts per episodes

In [105]:
data = pd.concat(collector)
data.drop('extra', 1, inplace = True)
data = data[data['speaker'].notnull()]
data = data[data['line'].notnull()]
data = data[data['speaker'] != data['line']]

In [123]:
data.to_csv('data/transcript_scraped.csv', index = False)